This program is used to estimate monthly oxygen flux pattern in the world's oceans.<br>
<br>

Histry: <br>
1. Modified by Zuchuan Li for revised model (2024-05-29)
2. Modified by Zuchuan Li for pretraining model (2024-09-24)

<br>
Authors: Zuchuan Li <br>
Date: 2022-09-21

In [125]:
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc
import calendar
import pickle
import glob
import os
import re
from scipy.ndimage import map_coordinates
import matplotlib.pyplot as plt

# 1.1. Paths to input data
chl_path = r"E:\2-Data\5-CHL"
par_path = r"E:\2-Data\7-PAR_8D"
sst_path = r"E:\2-Data\13-SST_8D"
sst_clm_path = r"E:\2-Data\2-AVHRR_SST"
slp_path = r"E:\2-Data\6-SLP"
wnd_path = r"E:\2-Data\8-CCMP"
mld_path = r"E:\2-Data\12-MLD"

# 1.2. Location and spatial resolution
nrows = 2160
ncols = 4320

intv = 180 / nrows
lon = np.linspace(-180 + intv / 2, 180 - intv / 2, ncols)
lat = np.linspace(90 - intv / 2, -90 + intv / 2, nrows)
LON, LAT = np.meshgrid(lon, lat)

LAT_sin = np.sin(LAT / 180 * np.pi)
LON_sin = np.sin(LON / 180 * np.pi) * np.cos(LAT / 180 * np.pi)
LON_cos =-np.cos(LON / 180 * np.pi) * np.cos(LAT / 180 * np.pi)

# 1.3. Wind latitude and longitude
wnd_nrows = 628
wnd_ncols = 1440
wnd_lat = np.linspace(-78.375, 78.375, wnd_nrows)
wnd_lon = np.linspace(0.125, 359.875, wnd_ncols)
wnd_lon = np.where(wnd_lon>180, wnd_lon-360, wnd_lon)

# 1.4. MLD and salinity climatologies
mld_fname = mld_path + r"\Argo_mixedlayers_monthlyclim_04142022.nc"
mlds_dt = nc.Dataset(mld_fname)

salts = np.array(mlds_dt['mls_dt'][:])
salts = np.where(salts < 0, np.nan, salts)
salts = np.flip(salts, axis=0)

mlds = np.array(mlds_dt['mld_dt_mean'][:])
mlds = np.where(mlds < 0, np.nan, mlds)
mlds = np.flip(mlds, axis=0)

# get a range of indeies within bounds
def get_valid_idx(val, itv, bound_low, bound_up):
    idx = np.arange(val - itv, val + itv + 1)
    return idx[(idx >= bound_low) & (idx <= bound_up)]

# missing value interpolation
def interp_missing_values(dt, step=1):
    dt_intp = np.copy(dt)
    idx_r, idx_c = np.where(np.isnan(dt))
    for j, k in zip(idx_r, idx_c):
        j_idx = get_valid_idx(j, step, 0, dt.shape[0]-1)
        k_idx = get_valid_idx(k, step, 0, dt.shape[1]-1)
        dt_intp[j, k] = np.nanmean(dt[j_idx,:][:,k_idx])
    return dt_intp

for i in range(mlds.shape[2]):
    mlds[:,:,i] = interp_missing_values(interp_missing_values(mlds[:,:,i]))
    salts[:,:,i] = interp_missing_values(interp_missing_values(salts[:,:,i]))
    
# resize the image
def my_img_re_2d(dt):
    r = np.linspace(0, dt.shape[0]-1, nrows)
    c = np.linspace(0, dt.shape[1]-1, ncols)
    cx, ry = np.meshgrid(c, r)
    rc = np.concatenate((ry[None,:,:], cx[None,:,:]), axis=0)
    return map_coordinates(dt, coordinates=rc, order=1, mode='nearest')

def my_img_re_3d(dt, date_first=False):
    dims = (nrows, ncols, dt.shape[0] if date_first else dt.shape[2])
    rs = np.zeros(dims)
    for m in range(rs.shape[2]):
        rs[:,:,m] = my_img_re_2d(dt[m,:,:]) if date_first else my_img_re_2d(dt[:,:,m])
    return rs

mlds = my_img_re_3d(mlds)
salts = my_img_re_3d(salts)


C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\3171239684.py:66: RuntimeWarning: Mean of empty slice
  dt_intp[j, k] = np.nanmean(dt[j_idx,:][:,k_idx])


In [150]:
import datetime

# show image
def visualize_img(dt, title=''):
    fig, axe = plt.subplots(nrows=1, ncols=1)
    axe.imshow(dt)
    axe.set_title(title)
    
    
# load parameter of ocean colors
def load_ocean_color(para, yr, yr_bg, path, rng, transpose=False, visualize=False):
    print('\tLoad {}...'.format(para))
    fname = para + '.' + str(yr) + str(yr_bg).rjust(3, '0') + '.hdf.npy'
    dt = np.load(path + "\\" + fname)
    print(path + "\\" + fname)
    
    dt = np.where((rng[0] <= dt) & (dt <= rng[1]), dt, np.nan)
    dt = np.transpose(dt) if transpose else dt
    print('\t', dt.shape)
    
    if visualize:
        visualize_img(dt, para)
    return dt


# Load wind product
def load_wind(yr_dy_bg, path, num_days_forward=16):
    wnds = None
    us = None
    vs = None
    for i in range(num_days_forward):
        # Construct file name
        wnd_date = yr_dy_bg + np.timedelta64(i, 'D')
        yr = pd.to_datetime(wnd_date).year
        mn = pd.to_datetime(wnd_date).month
        dy = pd.to_datetime(wnd_date).day
        ymd_str = str(yr) + str(mn).rjust(2, "0") + str(dy).rjust(2, "0")
        
        fn = path + "\\Y" + str(yr) + "\\M" + str(mn).rjust(2,"0") + "\\CCMP*Wind_Analysis_" + ymd_str + "*.nc"
        fn = glob.glob(fn)
        
        if len(fn) == 0:
            continue
        else:
            fn = fn[0]
            print('\t', fn)
        
        # Extract data
        dt = xr.open_dataset(fn)
        u = dt['uwnd'][:].to_numpy()
        v = dt['vwnd'][:].to_numpy()
        u[np.abs(u)>100] = np.nan
        v[np.abs(v)>100] = np.nan
        
        # Original data format:
        # Latitude ranges from -78.5 to 78.5
        # Longitude ranges from 0 to 360
        u = np.flip(u, axis=1)
        v = np.flip(v, axis=1)
        
        u_nt, u_rows, u_cols = u.shape
        cols_half = int(u_cols / 2)
        u = np.concatenate((u[:,:,cols_half:], u[:,:,:cols_half]), axis=2)
        v = np.concatenate((v[:,:,cols_half:], v[:,:,:cols_half]), axis=2)
        
        # Pad missing rows
        miss_rows = int((cols_half - u_rows) / 2)
        valid_idx = range(miss_rows, cols_half - miss_rows)
        
        u_2 = np.zeros((u_nt, cols_half, u_cols)) * np.nan
        v_2 = np.zeros((u_nt, cols_half, u_cols)) * np.nan
        u_2[:,valid_idx,:] = u
        v_2[:,valid_idx,:] = v
        
        wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
        u = np.nanmean(u_2, axis=0, keepdims=True)
        v = np.nanmean(v_2, axis=0, keepdims=True)
        
        wnds = wnd if wnds is None else np.concatenate((wnds, wnd), axis=0)
        us = u if us is None else np.concatenate((us, u), axis=0)
        vs = v if vs is None else np.concatenate((vs, v), axis=0)
    return us, vs, wnds
    

# shift days
def date_plus_days(yr, yr_dy_bg, shift):
    d_m8 = pd.to_datetime(str(yr) + "-01-01") + pd.Timedelta(yr_dy_bg + shift - 1, 'd')
    d_rf = d_m8 - pd.to_datetime(str(d_m8.year) + "-01-01")
    dy = 1 if d_m8.year > yr else d_rf.days + 1
    dy = 361 if d_m8.year < yr else dy
    return [d_m8.year, dy]
    
    
# --------------------- #
# load data
# --------------------- #
def load_data_4o2flux_pred(yr, yr_dy_bg, yr_dy_ed=None):
    print('\tYear', yr, '; Begin day', yr_dy_bg, '; End day', yr_dy_ed)
    
    # --------------------- #
    # 2.1. Year day
    # --------------------- #
    d_m8 = date_plus_days(yr, yr_dy_bg, -8)
    d_p8 = date_plus_days(yr, yr_dy_bg, 8)
    
    # --------------------- #
    # 2.2. Load Chl
    # --------------------- #
    chl = load_ocean_color('chl', yr, yr_dy_bg, chl_path, [0,100], transpose=True, visualize=False)
    chl_m8 = load_ocean_color('chl', d_m8[0], d_m8[1], chl_path, [0,100], transpose=True, visualize=False)
    chl_p8 = load_ocean_color('chl', d_p8[0], d_p8[1], chl_path, [0,100], transpose=True, visualize=False)
    #visualize_img(np.log(chl+1e-4), 'CHL')
    
    # --------------------- #
    # 2.3. Load PAR
    # --------------------- #
    par = load_ocean_color('par', yr, yr_dy_bg, par_path, [0,1000], transpose=True, visualize=False)
    par_m8 = load_ocean_color('par', d_m8[0], d_m8[1], par_path, [0,1000], transpose=True, visualize=False)
    par_p8 = load_ocean_color('par', d_p8[0], d_p8[1], par_path, [0,1000], transpose=True, visualize=False)
    
    # --------------------- #
    # 2.4. Load SST
    # --------------------- #
    sst = load_ocean_color('sst', yr, yr_dy_bg, sst_path, [-3,45], transpose=True, visualize=False)
    sst_m8 = load_ocean_color('sst', d_m8[0], d_m8[1], sst_path, [-3,45], transpose=True, visualize=False)
    sst_p8 = load_ocean_color('sst', d_p8[0], d_p8[1], sst_path, [-3,45], transpose=True, visualize=False)
    sst = sst if sst.shape[0] == nrows else my_img_re_2d(sst)
    sst_m8 = sst_m8 if sst_m8.shape[0] == nrows else my_img_re_2d(sst_m8)
    sst_p8 = sst_p8 if sst_p8.shape[0] == nrows else my_img_re_2d(sst_p8)
    
    # --------------------- #
    # 2.5. Wind speed 
    # --------------------- #
    print('\tLoad Wind data...')
    yr_dy_bg_2 = np.datetime64(str(yr) + "-01-01") + np.timedelta64(yr_dy_bg - 9, 'D')
    U, V, wnds = load_wind(yr_dy_bg_2, wnd_path, num_days_forward=16)
    
    # resize
    U = my_img_re_3d(U, date_first=True)
    V = my_img_re_3d(V, date_first=True)
    wnds = my_img_re_3d(wnds, date_first=True)
    #visualize_img(wnds[:,:,0], 'wnd')
    #visualize_img(U[:,:,0], 'U')
    #visualize_img(V[:,:,0], 'V')
    
    # --------------------- #
    # 2.6. Climatological MLD and salt
    # --------------------- #
    tmp = pd.to_datetime(str(yr) + "-01-01 00:00:00.0") + pd.Timedelta(yr_dy_bg - 1, 'd')
    mn = int(tmp.month)
    mld = np.copy(mlds[:,:,mn-1])
    salt = np.copy(salts[:,:,mn-1])
    #visualize_img(mld, 'MLD')
    #visualize_img(salt, 'SALT')
    
    # --------------------- #
    # 2.7. Climatological SST
    # --------------------- #
    fname = sst_clm_path + r"\clm.sst." + str(mn).rjust(2,"0") + ".npy"
    sst_clm = np.load(fname).T
    sst_clm = my_img_re_2d(sst_clm)
    #visualize_img(sst_clm, 'SST')
    
    # --------------------- #
    # 2.8. Combine variables
    # --------------------- #
    return {
        'SST': np.concatenate((sst_m8[:,:,None], sst[:,:,None], sst_p8[:,:,None]), axis=2),
        'CHL': np.concatenate((chl_m8[:,:,None], chl[:,:,None], chl_p8[:,:,None]), axis=2),
        'PAR': np.concatenate((par_m8[:,:,None], par[:,:,None], par_p8[:,:,None]), axis=2),
        'U': U,
        'V': V,
        "MLD_CLM": mld[:,:,None],
        "SAL_CLM": salt[:,:,None],
        "SST_CLM": sst_clm[:,:,None],
        }

# 2. Running

### 2.1. Run a given snapshot 

In [151]:
yr = 1998
yr_dy_bg = 1
yr_dy_ed = 8
X = load_data_4o2flux_pred(yr, yr_dy_bg, yr_dy_ed)


	Year 1998 ; Begin day 1 ; End day 8
	Load chl...
E:\2-Data\5-CHL\chl.1998001.hdf.npy
	 (2160, 4320)
	Load chl...
E:\2-Data\5-CHL\chl.1997361.hdf.npy
	 (2160, 4320)
	Load chl...
E:\2-Data\5-CHL\chl.1998009.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.1998001.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.1997361.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.1998009.hdf.npy
	 (2160, 4320)
	Load sst...
E:\2-Data\13-SST_8D\sst.1998001.hdf.npy
	 (2048, 4096)
	Load sst...
E:\2-Data\13-SST_8D\sst.1997361.hdf.npy
	 (2048, 4096)
	Load sst...
E:\2-Data\13-SST_8D\sst.1998009.hdf.npy
	 (2048, 4096)
	Load Wind data...
	 E:\2-Data\8-CCMP\Y1997\M12\CCMP_Wind_Analysis_19971224_V02.0_L3.0_RSS.nc


C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:75: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:76: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:77: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


	 E:\2-Data\8-CCMP\Y1997\M12\CCMP_Wind_Analysis_19971225_V02.0_L3.0_RSS.nc


C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:75: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:76: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:77: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


	 E:\2-Data\8-CCMP\Y1997\M12\CCMP_Wind_Analysis_19971226_V02.0_L3.0_RSS.nc


...

C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:75: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:76: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:77: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


	 E:\2-Data\8-CCMP\Y1998\M01\CCMP_Wind_Analysis_19980107_V02.0_L3.0_RSS.nc


C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:75: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:76: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:77: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


	 E:\2-Data\8-CCMP\Y1998\M01\CCMP_Wind_Analysis_19980108_V02.0_L3.0_RSS.nc


C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:75: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:76: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15440\593543715.py:77: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


## 2.2. Construct pretraining dataset

In [128]:
# get index of neighbor row/col
def get_neighbor_idx(val, itv, bound_low, bound_up):
    val = np.reshape(val, (-1, 1))
    idx = [val + i for i in range(-itv, itv + 1)]
    idx = np.concatenate(idx, axis=1)
    idx = np.where(idx < bound_up, idx, idx - bound_up)
    idx = np.where(idx >= bound_low, idx, idx + bound_up)
    return idx


# dt has 2/3 dimensions
# NOTE (@^@): the shape of the data
def extract_subset_sub(dt, row_idx, col_idx, rng):
    step = 2 * rng + 1
    rs = []
    for i in range(step):
        for j in range(step):
            tmp = np.transpose(dt[row_idx[:,i], col_idx[:,j], :])
            rs.append(tmp[None,:,:])
    rs = np.concatenate(rs, axis=0)
    rs = np.reshape(rs, (step, step, dt.shape[2], -1))
    return rs
    

# average the data in time domain
# designed specificly for given variables (U and V)
def resample_time(dt, cols):
    for col in cols:
        dt[col] = ((dt[col][:,:,range(0,16,4),:] + 
                    dt[col][:,:,range(1,16,4),:] + 
                    dt[col][:,:,range(2,16,4),:] + 
                    dt[col][:,:,range(3,16,4),:]) * 0.25)
    return dt


# build a sampling mask
def build_sampling_mask(dt, sub_cols, missing_count_thres=5, size=0.1):
    # missing values
    missing_count = 0
    for col in sub_cols:
        idx = np.isnan(dt[col]).astype(int)
        if dt[col].ndim > 2:
            idx = np.sum(idx, axis=2)
        missing_count += idx
    idx = missing_count <= missing_count_thres
    
    # Sample location
    idx_flat = np.reshape(idx, (np.prod(idx.shape),))
    size = int(idx.sum() * size)
    idx_sample = np.random.choice(np.where(idx_flat)[0], size=size, replace=False)
    
    idx_flat[:] = False
    idx_flat[idx_sample] = True
    return np.reshape(idx_flat, idx.shape)

        
# return subregion of data
# if dt is 3d, 3rd is time.
def extract_subset(dt, idx, cols, rng=1, row_bound=2160, col_bound=4320):
    row_idx, col_idx = np.where(idx)
    
    row_idx = get_neighbor_idx(row_idx, rng, 0, row_bound)
    col_idx = get_neighbor_idx(col_idx, rng, 0, col_bound)
    
    return {col:extract_subset_sub(dt[col], row_idx, col_idx, rng) 
            for col in cols}

    
# Sample the data
def sample_data(dt, sub_cols, rng=1, missing_count_thres=5, size=0.05, row_bound=2160, col_bound=4320):
    # build sampling mask
    mask = build_sampling_mask(dt, sub_cols, size=size, 
                               missing_count_thres=missing_count_thres)
    
    # Subset data
    dt_sub = extract_subset(dt, mask, dt.keys(), rng=rng, row_bound=row_bound, col_bound=col_bound)
    dt_sub['MASK'] = mask
    return dt_sub


In [6]:
import re
import os
import pickle

rs_path = r"C:\Projects\1-Projects\1-Oxygen\1-Program\pretraining_data"
cols = ['SST', 'CHL', 'PAR', 
        'MLD_CLM', 'SAL_CLM', 'SST_CLM']


# loop the year
num = 0
for yr in range(2003, 2022):
    yrdys = 366 if calendar.isleap(yr) else 365
    for dy in range(1, yrdys, 8):
        print(yr, dy)
        try:
            # load data
            dt = load_data_4o2flux_pred(yr, dy, min(dy+7,yrdys))
            
            # sample data
            dt_sub = sample_data(dt, cols, rng=1, missing_count_thres=5, size=0.1)
            
            # resample U and V
            dt_sub = resample_time(dt_sub, ['U', 'V'])
            
            # save
            fname = rs_path + "\\{}.{}.pickle".format(yr, str(dy).rjust(3,'0'))
            with open(fname, 'wb') as fid:
                pickle.dump(dt_sub, fid)
        except:
            print('Fail in calculation...')
    

2017 65
	Year 2017 ; Begin day 65 ; End day 72
	Load chl...
E:\2-Data\5-CHL\chl.2017065.hdf.npy
	 (2160, 4320)
	Load chl...
E:\2-Data\5-CHL\chl.2017057.hdf.npy
	 (2160, 4320)
	Load chl...
E:\2-Data\5-CHL\chl.2017073.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.2017065.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.2017057.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.2017073.hdf.npy
	 (2160, 4320)
	Load sst...
E:\2-Data\13-SST_8D\sst.2017065.hdf.npy
	 (2160, 4320)
	Load sst...
E:\2-Data\13-SST_8D\sst.2017057.hdf.npy
	 (2160, 4320)
	Load sst...
E:\2-Data\13-SST_8D\sst.2017073.hdf.npy
	 (2160, 4320)
	Load Wind data...
	 E:\2-Data\8-CCMP\Y2017\M02\CCMP_RT_Wind_Analysis_20170226_V02.1_L3.0_RSS.nc


C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:100: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:101: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:102: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


	 E:\2-Data\8-CCMP\Y2017\M02\CCMP_RT_Wind_Analysis_20170227_V02.1_L3.0_RSS.nc


C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:100: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:101: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:102: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


	 E:\2-Data\8-CCMP\Y2017\M02\CCMP_RT_Wind_Analysis_20170228_V02.1_L3.0_RSS.nc


...

	 E:\2-Data\8-CCMP\Y2020\M11\CCMP_RT_Wind_Analysis_20201106_V02.1_L3.0_RSS.nc


C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:100: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:101: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:102: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


	 E:\2-Data\8-CCMP\Y2020\M11\CCMP_RT_Wind_Analysis_20201107_V02.1_L3.0_RSS.nc


C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:100: RuntimeWarning: Mean of empty slice
  wnd = np.nanmean(np.sqrt(u_2**2 + v_2**2), axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:101: RuntimeWarning: Mean of empty slice
  u = np.nanmean(u_2, axis=0, keepdims=True)
C:\Users\zl59\AppData\Local\Temp\ipykernel_15848\6465438.py:102: RuntimeWarning: Mean of empty slice
  v = np.nanmean(v_2, axis=0, keepdims=True)


Fail in calculation...
2021 241
	Year 2021 ; Begin day 241 ; End day 248
	Load chl...
E:\2-Data\5-CHL\chl.2021241.hdf.npy
	 (2160, 4320)
	Load chl...
E:\2-Data\5-CHL\chl.2021233.hdf.npy
	 (2160, 4320)
	Load chl...
E:\2-Data\5-CHL\chl.2021249.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.2021241.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.2021233.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.2021249.hdf.npy
	 (2160, 4320)
	Load sst...
E:\2-Data\13-SST_8D\sst.2021241.hdf.npy
	 (2160, 4320)
	Load sst...
E:\2-Data\13-SST_8D\sst.2021233.hdf.npy
	 (2160, 4320)
	Load sst...
Fail in calculation...
2021 249
	Year 2021 ; Begin day 249 ; End day 256
	Load chl...
E:\2-Data\5-CHL\chl.2021249.hdf.npy
	 (2160, 4320)
	Load chl...
E:\2-Data\5-CHL\chl.2021241.hdf.npy
	 (2160, 4320)
	Load chl...
E:\2-Data\5-CHL\chl.2021257.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.2021249.hdf.npy
	 (2160, 4320)
	Load par...
E:\2-Data\7-PAR_8D\par.2021241.hdf.npy
	

# Sand box

In [ ]:
# load sea level pressure data
def load_slp_data(yr, path):
    # Load two year data
    slps = None
    slp_dates = None
    slp_fnames = ['slp.' + str(yr-1) + '.nc', 
                  'slp.' + str(yr) + '.nc', 
                  'slp.' + str(yr+1) + '.nc']
    
    for fn in slp_fnames:
        dt = xr.open_dataset(path + "\\" + fn)
        print(slp_path + "\\" + fn)
        slp = dt['slp'][:].to_numpy()
        slp = (slp[0::4,:,:] + slp[1::4,:,:] + slp[2::4,:,:] + slp[3::4,:,:]) / 4
        slps = slp if slps is None else np.concatenate((slps, slp), axis=0)
        
        date = dt['time'][::4].to_numpy()
        slp_dates = date if slp_dates is None else np.concatenate((slp_dates, date), axis=0)
        
    # Since longitude is from 0 to 360, should be changed to [-180, 180]
    slp_cols_half = int(slps.shape[2] / 2)
    slps = np.concatenate((slps[:,:,slp_cols_half:], slps[:,:,0:slp_cols_half]), axis=2)
    return slps, slp_dates


# --------------------- #
# 2.5. Load SLP
# --------------------- #
    """
    print('\tLoad SLP...')
    if not (yr == pre_yr):
        global slps, slp_dates
        slps, slp_dates = load_slp_data(yr, slp_path)
        
    # Time horizon
    yr_dy_bg_2 = np.datetime64(str(yr) + "-01-01") + np.timedelta64(yr_dy_bg - 9, 'D')
    yr_dy_ed_2 = np.datetime64(str(yr) + "-01-01") + np.timedelta64(yr_dy_bg + 7 - 1, 'D')
    slp_date_idx = (yr_dy_bg_2 <= slp_dates) & (slp_dates <= yr_dy_ed_2)
    
    # Resize
    slps_sub = my_img_re_3d(slps[slp_date_idx,:,:], date_first=True)
    #visualize_img(slps_sub[:,:,0], 'slp')
    """
    
# --------------------- #
    # 2.7. Heat flux
    # --------------------- #
    """
    dys = pd.to_datetime(str(yr) + "-01-01 00:00:00.0") - pd.to_datetime("1900-01-01 00:00:00.0")
    dys = (dys + pd.Timedelta(yr_dy_bg, 'd')).days
    idx = np.argmin(np.abs(hf_mns - dys))
    lhf = my_img_re_2d(slhf[idx,:,:])
    shf = my_img_re_2d(sshf[idx,:,:])
    #visualize_img(lhf, 'LHF')
    #visualize_img(shf, 'SHF')
    """